# Ax optimalization framework [TODO]

Ax is an open-source package from PyTorch that helps you find a minima for any function over the range of parameters. One of the useful ML applications is to find the best hyperparameters for training a model to achieve minimal loss.

Sources:
- https://ax.dev/docs/core.html
- https://github.com/facebook/Ax/blob/master/tutorials/building_blocks.ipynb

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Ax Optimalization Framework
Adaptive experimentation is the machine-learning guided process of iteratively exploring a (possibly infinite) parameter space in order to identify optimal configurations in a resource-efficient manner. Bayesian optimization is powered by BoTorch.

Source:
- https://github.com/facebook/Ax

In [2]:
from ax import *

In [3]:
# Define experiment input parameters.
range_param1 = RangeParameter(name="x1", lower=0.0, upper=10.0, parameter_type=ParameterType.FLOAT)
range_param2 = RangeParameter(name="x2", lower=0.0, upper=10.0, parameter_type=ParameterType.FLOAT)

# Define experiment constraints over parameters.
# Sum constraints enforce that the sum of a set of parameters is greater or less than some bound, 
# and order constraints enforce that one parameter is smaller than the other.
sum_constraint = SumConstraint(
    parameters=[range_param1, range_param2], 
    is_upper_bound=True, 
    bound=5.0,
)

order_constraint = OrderConstraint(
    lower_parameter = range_param1,
    upper_parameter = range_param2,
)

# Create search space and register parameters and constrains
search_space = SearchSpace(
    parameters=[range_param1, range_param2],
    parameter_constraints=[sum_constraint, order_constraint]
)

#choice_param = ChoiceParameter(name="choice", values=["foo", "bar"], parameter_type=ParameterType.STRING)
#fixed_param = FixedParameter(name="fixed", value=[True], parameter_type=ParameterType.BOOL)

In [4]:
# Define experiment
experiment = Experiment(
    name="Demo experiment",
    search_space=search_space
)

In [5]:
# Generate arms as assignments of parameters to values, that lie within the search space
sobol = Models.SOBOL(search_space=experiment.search_space)
# generate 5 points
generator_run = sobol.gen(5)

# show generated inputs
for arm in generator_run.arms:
    print(arm)

Arm(parameters={'x1': 0.5808260291814804, 'x2': 1.6990402340888977})
Arm(parameters={'x1': 1.111799106001854, 'x2': 2.5313133001327515})
Arm(parameters={'x1': 0.27351927012205124, 'x2': 4.42659854888916})
Arm(parameters={'x1': 1.287938803434372, 'x2': 2.460074871778488})
Arm(parameters={'x1': 0.8459854125976562, 'x2': 1.0500745475292206})


In [6]:
# An optimization config is composed of an objective metric to be minimized or maximized in the experiment, 
# and optionally a set of outcome constraints that place restrictions on how other metrics can be
# moved by the experiment.

class BoothMetric(Metric):
    # Generate model metrics for provided arms
    def fetch_trial_data(self, trial):
        records = []
        for arm_name, arm in trial.arms_by_name.items():
            params = arm.parameters
            records.append({
                "arm_name": arm_name,
                "metric_name": self.name,
                "mean": (params["x1"] + 2*params["x2"] - 7)**2 + (2*params["x1"] + params["x2"] - 5)**2,
                "sem": 0.0,
                "trial_index": trial.index,
            })
            
        print ('Evaluated data size:', len(records))
        return Data(df=pd.DataFrame.from_records(records))

In [7]:
optimization_config = OptimizationConfig(
    objective = Objective(
        metric=BoothMetric(name="booth"), 
        minimize=True,
    ),
)

experiment.optimization_config = optimization_config

In [8]:
# Before an experiment can collect data, it must have a Runner attached. 
# A runner handles the deployment of trials.
# A trial must be "run" before it can be evaluated.

class ExperimentRunner(Runner):
    def run(self, trial):
        print ('ExperimentRunner is running, trial:', trial)
        # e.g. deploy data to production
        return {"name": str(trial.index)}
    
experiment.runner = ExperimentRunner()

In [9]:
# Now we can collect data for arms within our search space and begin the optimization.
# - Generating arms for an initial exploratory batch (already done above, using Sobol)
# - Adding these arms to a trial
# - Running the trial
# - Evaluating the trial
# - Generating new arms based on the results, and repeating

experiment.new_batch_trial(generator_run=generator_run)

BatchTrial(experiment_name='Demo experiment', index=0, status=TrialStatus.CANDIDATE)

In [10]:
for arm in experiment.trials[0].arms:
    print(arm)

Arm(name='0_0', parameters={'x1': 0.5808260291814804, 'x2': 1.6990402340888977})
Arm(name='0_1', parameters={'x1': 1.111799106001854, 'x2': 2.5313133001327515})
Arm(name='0_2', parameters={'x1': 0.27351927012205124, 'x2': 4.42659854888916})
Arm(name='0_3', parameters={'x1': 1.287938803434372, 'x2': 2.460074871778488})
Arm(name='0_4', parameters={'x1': 0.8459854125976562, 'x2': 1.0500745475292206})


In [11]:
# Extend arms
experiment.new_trial().add_arm(Arm(name='single_arm', parameters={'x1': 1, 'x2': 1}))

Trial(experiment_name='Demo experiment', index=1, status=TrialStatus.CANDIDATE, arm=Arm(name='single_arm', parameters={'x1': 1, 'x2': 1}))

In [13]:
from ax.core.base_trial import TrialStatus

for i in range(0, len(experiment.trials)):
    trial = experiment.trials[i]
    if trial.status == TrialStatus.CANDIDATE:
        experiment.trials[i].run()

ExperimentRunner is running, trial: BatchTrial(experiment_name='Demo experiment', index=0, status=TrialStatus.CANDIDATE)
ExperimentRunner is running, trial: Trial(experiment_name='Demo experiment', index=1, status=TrialStatus.CANDIDATE, arm=Arm(name='single_arm', parameters={'x1': 1, 'x2': 1}))


In [14]:
data = experiment.fetch_data()
data.df

Evaluated data size: 5
Evaluated data size: 1


,arm_name,metric_name,mean,sem,trial_index
0,0_0,booth,13.703643,0.0,0
1,0_1,booth,0.741641,0.0,0
2,0_2,booth,4.523618,0.0,0
3,0_3,booth,0.628416,0.0,0
4,0_4,booth,21.532185,0.0,0
5,single_arm,booth,20.000000,0.0,1


In [15]:
# Now we can model the data collected for the initial set of arms via Bayesian optimization 
# (using the Botorch model default of Gaussian Process with Expected Improvement acquisition function) 
# to determine the new arms for which to fetch data next.

gpei = Models.BOTORCH(experiment=experiment, data=data)
# Generate 5 points
generator_run = gpei.gen(5)
best_arm, _ = generator_run.best_arm_predictions
experiment.new_batch_trial(generator_run=generator_run)
# ...

best_parameters = best_arm.parameters

c:\apps\python36\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning:

A not p.d., added jitter of 1e-08 to the diagonal

c:\apps\python36\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning:

A not p.d., added jitter of 1e-08 to the diagonal

c:\apps\python36\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning:

A not p.d., added jitter of 1e-08 to the diagonal

c:\apps\python36\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning:

A not p.d., added jitter of 1e-08 to the diagonal



In [16]:
best_parameters

{'x1': 1.287938803434372, 'x2': 2.460074871778488}

In [17]:
# run another trial
for i in range(0, len(experiment.trials)):
    trial = experiment.trials[i]
    if trial.status == TrialStatus.CANDIDATE:        
        experiment.trials[i].run()

data = experiment.fetch_data()
data.df

ExperimentRunner is running, trial: BatchTrial(experiment_name='Demo experiment', index=2, status=TrialStatus.CANDIDATE)
Evaluated data size: 5
Evaluated data size: 1
Evaluated data size: 5


,arm_name,metric_name,mean,sem,trial_index
0,0_0,booth,13.703643,0.0,0
1,0_1,booth,0.741641,0.0,0
2,0_2,booth,4.523618,0.0,0
3,0_3,booth,0.628416,0.0,0
4,0_4,booth,21.532185,0.0,0
5,single_arm,booth,20.000000,0.0,1
6,2_0,booth,6.500000,0.0,2
7,2_1,booth,4.501274,0.0,2
8,2_2,booth,4.864864,0.0,2
9,2_3,booth,2.101612,0.0,2


In [18]:
# At any point, we can also save our experiment to a JSON file. To ensure that our custom metrics 
# and runner are saved properly, we first need to register them.

from ax.storage.metric_registry import register_metric
from ax.storage.runner_registry import register_runner

register_metric(BoothMetric)
register_runner(ExperimentRunner)

save(experiment, "experiment.json")

loaded_experiment = load("experiment.json")

In [19]:
# Gaussian Processes (GPs) are used for Bayesian Optimization in Ax, the get_GPEI function constructs 
# a model that fits a GP to the data, and uses the EI acquisition function to generate new points on calls to gen.
# This code fits a GP and generates a batch of 5 points which maximizes EI:

from ax.modelbridge.factory import get_GPEI

model = get_GPEI(experiment, data)
generator_run_2 = model.gen(n=5, optimization_config=optimization_config)
best_arm, _ = generator_run_2.best_arm_predictions

best_parameters = best_arm.parameters
print ('best_parameters:', best_parameters)

[INFO 05-05 16:08:11] ModelBridge: Leaving out out-of-design observations for arms: 2_3, 2_1, 2_2, 2_0
c:\apps\python36\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning:

A not p.d., added jitter of 1e-08 to the diagonal

c:\apps\python36\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning:

A not p.d., added jitter of 1e-08 to the diagonal

c:\apps\python36\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning:

A not p.d., added jitter of 1e-08 to the diagonal

c:\apps\python36\lib\site-packages\gpytorch\utils\cholesky.py:44: NumericalWarning:

A not p.d., added jitter of 1e-08 to the diagonal



best_parameters: {'x1': 1.287938803434372, 'x2': 2.460074871778488}


In [20]:
# We make predictions by constructing a list of ObservationFeatures objects with the parameter 
# values for which we want predictions

from ax.core.observation import ObservationFeatures

obs_feats = [
    ObservationFeatures(parameters={'x1': 3.14, 'x2': 2.72}),
    ObservationFeatures(parameters={'x1': 1.41, 'x2': 1.62}),
]


# The output of predict is the mean estimate of each metric and the covariance (across metrics) for each point.
# Make model predictions (mean and covariance) for the given observation features.
# Predictions are made for all outcomes. If an out-of-design observation can successfully be transformed,
# the predicted value will be returned. Othwerise, we will attempt to find that observation in the training data
# and return the raw value.

f, cov = model.predict(obs_feats)

In [21]:
f

{'booth': [1.2811189283872046, 7.25122716155486]}

In [22]:
cov

{'booth': {'booth': [103.12604404233777, 2.778389333951025]}}

In [ ]:
# from ax.plot.slice import plot_slice
# from ax.utils.notebook.plotting import render, init_notebook_plotting

# init_notebook_plotting()
# render(plot_slice(
#     model=m,
#     param_name='x1',  # slice on values of 'x1'
#     metric_name='metric_a',
#     slice_values={'x2': 7.5},  # Fix at this value for the slice
# ))